In [1]:
## imports for custom overlay
from pynq import Overlay
## Import for DMA alloc
from pynq import allocate
## import for video driver API support
from pynq.lib.video import *
## import for timing
import time
## import for rectangles and any opencv we do in software
import cv2

## CUSTOM OVERLAY 
hudson_overlay = Overlay('hudson_final_thresh.bit')
# DMA vars
vdma = hudson_overlay.video.axi_vdma
dma = hudson_overlay.redDMA.axi_dma_0
# config axi lite access
axilite = hudson_overlay.redDMA.threshold_accel_0

In [2]:
# send input buffer and read the output

def run_kernel():
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()

In [3]:
# Set HDMI in and out vars
hdmi_in = hudson_overlay.video.hdmi_in
hdmi_out = hudson_overlay.video.hdmi_out
# initialize IO
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)



In [4]:
hdmi_in.start()
hdmi_out.start()

In [5]:
# Read in a frame and allocate space for it in
# what will become axi stream buffer
shaped_in = hdmi_in.readframe()
in_buffer = allocate(shape=(720,1280,3), dtype=np.uint8)
out_buffer = allocate(shape=(720,1280,3), dtype=np.uint8)

# show on the screen what an in buffer frame looks like
in_buffer[:] = shaped_in
hdmi_out.writeframe(in_buffer)

In [6]:
# set axi lite vals
axilite.write(0x10, 30) # thresh
axilite.write(0x18, 255) # maxval
axilite.write(0x20, 720) # rows
axilite.write(0x28, 1280) # cols
# axilite.write(0x40, 0) # sigma value for gaus blur

In [7]:
# point HDMI out to the in buffer
hdmi_out.writeframe(in_buffer)

size = 200

numframes = 600
start = time.time()

# time <numframes> frames of processing
for _ in range(numframes):
    # read a frame into buffer to hold it
    in_buffer = hdmi_in.readframe()
    # send in buffer to hardware for processing
    run_kernel()
    # read the axi lite ports for rectangle coordinates and put them on the in_buffer
    cv2.rectangle(in_buffer, (axilite.read(0x30),axilite.read(0x38)), (size,size), (0, 0, 255), 2)
    # write buffer to screen
    # NOTE: change this line to write out buffer to see what the processed image looks like
    #       or in buffer to see the original image with recatagles drawn
    hdmi_out.writeframe(out_buffer)
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

    

Frames per second:  23.463065025879057


almost 24 frames per second when thresholding in hardware

In [8]:
## Software thresholding test
gray = np.ndarray(shape=(hdmi_in.mode.height, 
                              hdmi_in.mode.width,3), dtype=np.uint8)

buffer1 = allocate(shape=(720,1280,3), dtype=np.uint8)
buffer2 = allocate(shape=(720,1280,3), dtype=np.uint8)

numframes = 300
start = time.time()

for _ in range(numframes):
    buffer1 = hdmi_in.readframe()
    cv2.threshold(buffer1,30,255,cv2.THRESH_BINARY, dst=buffer2)
    hdmi_out.writeframe(buffer2)
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

Frames per second:  37.85172979975052


almost 38 frames per second when thresholding in software

In [9]:
hdmi_in.close()
hdmi_out.close()